# Job Placement Predictor

This project builds a model that predicts whether a student can get job 
placement opportunity after graduating based on his/her academic performance, work experiences, projects etc. 

## Project Work Flow

1. Load the data into a dataframe
2. Perform Data Preprocessing like handling missing values, feature creation etc.
3. Perform Eploratory Data Analysis and get valuable insights from the data
4. Perform feature selection and select the best algorithm which fits the data
5. Save the model in a pickle file

In [1]:
# import libriaries
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
# load data set
df = pd.read_csv("Placement_Data_Full_Class.csv")
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


## Data Preprocessing

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sl_no           215 non-null    int64  
 1   gender          215 non-null    object 
 2   ssc_p           215 non-null    float64
 3   ssc_b           215 non-null    object 
 4   hsc_p           215 non-null    float64
 5   hsc_b           215 non-null    object 
 6   hsc_s           215 non-null    object 
 7   degree_p        215 non-null    float64
 8   degree_t        215 non-null    object 
 9   workex          215 non-null    object 
 10  etest_p         215 non-null    float64
 11  specialisation  215 non-null    object 
 12  mba_p           215 non-null    float64
 13  status          215 non-null    object 
 14  salary          148 non-null    float64
dtypes: float64(6), int64(1), object(8)
memory usage: 25.3+ KB


There are two types of data types in the data set. The `int64` and `object` data sets.

We can see that the `salary` column has null values.

We will be separating the categorical and numerical columns.

In [6]:
# Seperating te categorical and numerical columns
cat_col = list(df.select_dtypes(include=["object"]).columns)
num_col = list(df.select_dtypes(exclude=["object"]).columns)

print(cat_col, num_col)

['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status'] ['sl_no', 'ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']


In [7]:
# check for null values
df.isnull().sum()

sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

The salary column has 67 null values.

### Handling Null values

In [9]:
# view the shape of the data frame
df.shape

(215, 15)

In [10]:
# write a function that plots distribution to help us fill in the missing
# values
def plot_kedplot(col):
    
    plt.figure(figsize=(10, 7))
    sns.kdeplot(df["salary"], label="Actual Salary", color="orange")
    sns.kdeplot(df[col], color="black", label=col)
    plt.legend()

In [11]:
# get the mode frequent salary value
df["salary"].mode()

0    300000.0
Name: salary, dtype: float64

In [24]:
(dict(df["salary"].value_counts()).items())[0]

TypeError: 'dict_items' object is not subscriptable